In [3]:
import pandas as pd
import csv
import re
import string
import nltk

from pythainlp.tokenize import Tokenizer
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import dict_trie

from nltk.corpus import words
from stop_words import get_stop_words

In [4]:
# import data from csv
file_name = 'category1'

FILE = './Data/{}.csv'.format(file_name)
df = pd.read_csv(FILE)

In [5]:
df.head()

,Unnamed: 0,category_0_0,category_0_1,category_0_2,category_0_3,category_0_4,category_0_5,category_0_6,category_0_7,category_0_8,...,category_33_234,category_33_235,category_33_236,category_33_237,category_33_238,category_33_239,category_33_240,category_33_241,category_33_242,category_33_243
0,0,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,...,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New
1,1,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,...,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต
2,2,Samsung Galaxy M30s (4/64GB) ( โทรศัพท์มือถือ ),โทรศัพท์มือถือ Samsung Galaxy A50s Ram4/64GB (...,Samsung Galaxy A10s (RAM2GB/ROM32GB),Samsung Galaxy Note 10 + (12/256 GB) ( โทรศัพท...,Samsung Galaxy A7 (2017),Samsung Galaxy A9 (2018) 6 GB RAM 128 GB ROM 6...,Samsung A30S (RAM 4GB / ROM 64 GB) มือ 1 ประกั...,(มือ1 ศูนย์ไทย) Samsung Galaxy Note 10 Plus No...,Samsung Galaxy A70 (8/128GB) ( โทรศัพท์มือถือ ),...,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New
3,3,แบตอึดอยู่นาน สนุกได้ทั้งวัน กับความจุ 6000 mA...,GSM 850/900/1800/1900MHz,สมาร์ทโฟน (โทรศัพท์มือถือพร้อมระบบปฏิบัติการ),ขนาดหน้าจอ 6.8 นิ้ว,ระบบเซ็นเซอร์ (Sensor)\r\n- ระบบตรวจสอบลายนิ้ว...,จอแสดงผล Super AMOLED 24-bit (16 ล้านสี)Fullvi...,คุณสมบัติ Samsung A30S,Samsung Galaxy Note10 plus TH ประกันศูนย์ไทย 1...,Galaxy A70 (SM-A705FZBGTHL),...,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New
4,4,ระบบกล้องหลัง 3 ตัว ที่มีกล้องชัดสูงสุดถึง 48 ...,3G WCDMA 850/900/1900/2100MHz,จอแสดงผล TFT-LCD 24-bit (16 ล้านสี)- หน้าจอทรง...,Ram 12 GB,คุณสมบัติการกันน้ำ (Waterproof)\r\n- กันน้ำได้...,ระบบตรวจสอบลายนิ้วมือ (Fingerprint)ระบบจดจำใบห...,จอแสดงผล Super AMOLED 24-bit (16 ล้านสี),แบรนด์ ซัมซุง,สเปคเทพของเกมเมอร์,...,GloryStar BASEUS PUBG ไฟฟรีเกมที่วางโทรศัพท์แผ...,GloryStar นาฬิกาติดตามตำแหน่งอัจฉริยะเด็กนาฬิก...,GloryStar Joy-CON ตัวควบคุมเกมแป้นเกมส์จอยเกมส...,GloryStar คอนโซลเกมคลาสสิก 8 BitsProfessional ...,GloryStar สีน้ำเงินแปรงเกมควบคุมเครื่องรับสัญญ...,GloryStar Flysky รีโมทคอนโทรล FS-IT4S + IA4B 4...,Top-Sky PlayStation 2 PS2 Memory Card - 32MB,Top-Sky PlayStation 2 PS2 Memory Card - 256MB,Top-Sky PlayStation 2 PS2 Memory Card - 128MB,Top - Sky 3 - in - 1 PS4/PS4 Slim/PS4 เกมโปรที...


In [6]:
# Drop a column that not use
data = df.drop(columns=['Unnamed: 0'])

# Drop column that all equal to NaN
data = data.dropna(axis=1, how='all')

# Fill NaN with "-"
data = data.fillna('-')

In [7]:
data.head()

,category_0_0,category_0_1,category_0_2,category_0_3,category_0_4,category_0_5,category_0_6,category_0_7,category_0_8,category_0_9,...,category_33_234,category_33_235,category_33_236,category_33_237,category_33_238,category_33_239,category_33_240,category_33_241,category_33_242,category_33_243
0,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,...,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New,สมาร์ทดีไวซ์ New
1,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,Smartphones,...,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต,แก็ดเจ็ต
2,Samsung Galaxy M30s (4/64GB) ( โทรศัพท์มือถือ ),โทรศัพท์มือถือ Samsung Galaxy A50s Ram4/64GB (...,Samsung Galaxy A10s (RAM2GB/ROM32GB),Samsung Galaxy Note 10 + (12/256 GB) ( โทรศัพท...,Samsung Galaxy A7 (2017),Samsung Galaxy A9 (2018) 6 GB RAM 128 GB ROM 6...,Samsung A30S (RAM 4GB / ROM 64 GB) มือ 1 ประกั...,(มือ1 ศูนย์ไทย) Samsung Galaxy Note 10 Plus No...,Samsung Galaxy A70 (8/128GB) ( โทรศัพท์มือถือ ),Smartphone โทรศัพท์มือถือ Samsung Galaxy A10S ...,...,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New,เครื่องเล่นเกมคอมโซล New
3,แบตอึดอยู่นาน สนุกได้ทั้งวัน กับความจุ 6000 mA...,GSM 850/900/1800/1900MHz,สมาร์ทโฟน (โทรศัพท์มือถือพร้อมระบบปฏิบัติการ),ขนาดหน้าจอ 6.8 นิ้ว,ระบบเซ็นเซอร์ (Sensor)\r\n- ระบบตรวจสอบลายนิ้ว...,จอแสดงผล Super AMOLED 24-bit (16 ล้านสี)Fullvi...,คุณสมบัติ Samsung A30S,Samsung Galaxy Note10 plus TH ประกันศูนย์ไทย 1...,Galaxy A70 (SM-A705FZBGTHL),แบรนด์ ซัมซุง,...,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New,คอนโซล New
4,ระบบกล้องหลัง 3 ตัว ที่มีกล้องชัดสูงสุดถึง 48 ...,3G WCDMA 850/900/1900/2100MHz,จอแสดงผล TFT-LCD 24-bit (16 ล้านสี)- หน้าจอทรง...,Ram 12 GB,คุณสมบัติการกันน้ำ (Waterproof)\r\n- กันน้ำได้...,ระบบตรวจสอบลายนิ้วมือ (Fingerprint)ระบบจดจำใบห...,จอแสดงผล Super AMOLED 24-bit (16 ล้านสี),แบรนด์ ซัมซุง,สเปคเทพของเกมเมอร์,SKU 571250946_TH-1048780375,...,GloryStar BASEUS PUBG ไฟฟรีเกมที่วางโทรศัพท์แผ...,GloryStar นาฬิกาติดตามตำแหน่งอัจฉริยะเด็กนาฬิก...,GloryStar Joy-CON ตัวควบคุมเกมแป้นเกมส์จอยเกมส...,GloryStar คอนโซลเกมคลาสสิก 8 BitsProfessional ...,GloryStar สีน้ำเงินแปรงเกมควบคุมเครื่องรับสัญญ...,GloryStar Flysky รีโมทคอนโทรล FS-IT4S + IA4B 4...,Top-Sky PlayStation 2 PS2 Memory Card - 32MB,Top-Sky PlayStation 2 PS2 Memory Card - 256MB,Top-Sky PlayStation 2 PS2 Memory Card - 128MB,Top - Sky 3 - in - 1 PS4/PS4 Slim/PS4 เกมโปรที...


In [35]:
data_category_0 = data.filter(regex='category_0', axis=1)

# Clean text

In [36]:
# clean the text
def clean_text_category0(msg):
    msg = str(msg).lower()
    if re.match("[0-9]\*[0-9]",str(msg)):
        msg = re.sub(r"\*","x",str(msg))
        
    msg = re.sub(r"\&","และ",str(msg))
    msg = re.sub(r"\%","เปอร์เซนต์",str(msg))
    msg = re.sub(r"แบรนด์","brand",str(msg))
    msg = re.sub(r"แรม"," ram ",str(msg))
    msg = re.sub(r"รอม"," rom ",str(msg))
    msg = re.sub(r"[b]attery\s?[c]apacity","ความจุแบตเตอรี่",str(msg))
    msg = re.sub(r"\฿,"," บาท ",str(msg))
    msg = re.sub(r"[\d\s]*มิลลิเมตร|mm|millimeter\.?"," มม. ",str(msg))
    msg = re.sub(r"[\d\s]*เซนติเมตร|cm|centimeter\.?"," ซม. ",str(msg))
    msg = re.sub(r"\&","และ",str(msg))
    msg = re.sub(r"\,","",str(msg))
    msg = re.sub(r"[^a-z0-9ก-๙\.\s]"," ", str(msg))
    msg = re.sub(r"\s+"," ",str(msg))
    
    return msg

In [37]:
# Cleaning text with clean_text function and keep in dict
data_clean = {}
for key,value in data_category_0.iteritems():
    tmp = [clean_text_category0(txt) for txt in value if not txt == '-']
    data_clean[key] = tmp 

In [38]:
# Show some of data in dict
dict(list(data_clean.items())[0:10])

{'category_0_0': ['โทรศัพท์มือถือและแท็บเล็ต',
  'smartphones',
  'samsung galaxy m30s 4 64gb โทรศัพท์มือถือ ',
  'แบตอึดอยู่นาน สนุกได้ทั้งวัน กับความจุ 6000 mah และสามารถชาร์จแบตเร็ว ด้วย fast charging',
  'ระบบกล้องหลัง 3 ตัว ที่มีกล้องชัดสูงสุดถึง 48 mp มาพร้อมกับเลนส์มุมกว้างพิเศษ ultra wide',
  'ใช้ exynos 9611 สเป็กแรงขึ้นเล่นเกมส์ได้ไม่มีสะดุด กับ ram 4 gb rom 64 gb',
  'หน้าจอแสดงผล super amoled infinity u ขนาด 6.4 นิ้ว',
  'brand ซัมซุง',
  'sku 480942616 th 870640886',
  'ประเภทของการรับประกัน มีการรับประกัน',
  'model galaxy m30s 4 64gb ',
  'ระยะเวลาการรับประกัน 1 ปี',
  '793'],
 'category_0_1': ['โทรศัพท์มือถือและแท็บเล็ต',
  'smartphones',
  'โทรศัพท์มือถือ samsung galaxy a50s ram4 64gb รับประกันศูนย์ by jaymart',
  'gsm 850 900 1800 1900mhz',
  '3g wcdma 850 900 1900 2100mhz',
  '4g fdd lte b1 2100 b3 1800 b5 850 b7 2600 b8 900 b20 800 b28 700 ',
  '4g tdd lte b38 2600 b40 2300 b41 2500 ',
  'รองรับ 2 ซิม nano sim',
  'หน่วยประมวลผล exynos 9610 quad 2.3 ghz quad 1.7 ghz

# Capture attribute of product by regex

def cap_attr(text):
    m = re.match("([ก-๙A-Za-z]+)\:\s?([ก-๙A-Za-z0-9]+)",text)
    if m:
        yield text
    elif re.match("SKU\s\d+\_TH\-\d+",text):
        yield text
    else:
        m = re.search("([ก-๙A-Za-z]+)\:\s?([ก-๙A-Za-z0-9]+)",text)
        if m is not None:
            p = re.findall("([^:\s]*):\s?([^:\s]*)",text)
            yield p

attr = {}
for i,v in data_clean.items():
    tmp_list = []
    for txt in v:
        match = cap_attr(txt)
        for x in match:
            tmp_list.append(x)
    attr[i] = tmp_list

attr

attr_list = {}
for i,v in attr.items():
    tmp_list = []
    for t in v:
        if type(t) != list:
            t_split = t.split(':')
            if len(t_split)<3:
                tmp_list.append(t_split)
        else:
            for ele in t:
                if not (ele[0]!='' or ele[1]!=''):
                    res = [list(ele)]
            for l in res:
                tmp_list.append(l)
    attr_list[i] = tmp_list

attr_list

dict_attr = {}
dict_value = {}
for i,v in attr_list.items():
    tmp_attr = []
    tmp_value = []
    for e in v:
        tmp_attr.append(e[0])
        tmp_value.append(e[1])
    dict_attr[i] = tmp_attr
    dict_value[i] = tmp_value

dict_value

# Proprocessing text

In [26]:
# download stopwords of TH and EN
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))

# path of custom dictionary for word tokenize
PATH_TO_CUSTOM_DICT = './Data/custom_dict.txt'

# split text to words 
def split_word(text):
    
    # Split text to words
    _tokenizer = Tokenizer(custom_dict=PATH_TO_CUSTOM_DICT, engine='newmm')
    words = _tokenizer.word_tokenize(text)

    # Remove stop words in TH and EN
    words = [i for i in words if not i in th_stop and not i in en_stop]
    
    # Delete whitespace
    words = [i for i in words if not ' ' in i]

    return words

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


## Tokenize text

In [27]:
token = {}
for key,value in data_clean.items():
    tmp = [split_word(txt) for txt in value]
    token[key] = tmp

In [28]:
dict(list(token.items())[0:10])

{'category_0_0': [['โทรศัพท์มือถือและแท็บเล็ต'],
  ['smartphones'],
  ['samsung', 'galaxy', 'm', '30', 's', '4', '64', 'gb', 'โทรศัพท์มือถือ'],
  ['แบต',
   'อึดอยู่นาน',
   'สนุกได้ทั้งวัน',
   'กับความจุ',
   '6000',
   'mah',
   'และสามารถชาร์จ',
   'แบต',
   'fast',
   'charging'],
  ['ระบบ',
   'กล้องหลัง',
   '3',
   'ตัว',
   'ที่มีกล้องชัดสูงสุดถึง',
   '48',
   'mp',
   'มาพร้อมกับเลนส์มุมกว้างพิเศษ',
   'ultra',
   'wide'],
  ['exynos',
   '9611',
   'สเป็กแรงขึ้นเล่นเกมส์ได้ไม่มีสะดุด',
   'ram',
   '4',
   'gb',
   'rom',
   '64',
   'gb'],
  ['หน้าจอแสดงผล', 'super', 'amoled', 'infinity', 'u', 'ขนาด', '6.4', 'นิ้ว'],
  ['brand', 'ซัมซุง'],
  ['sku', '480942616', 'th', '870640886'],
  ['ประเภทของการรับประกัน', 'มีการรับประกัน'],
  ['model', 'galaxy', 'm', '30', 's', '4', '64', 'gb'],
  ['ระยะเวลาการรับประกัน', '1', 'ปี'],
  ['793']],
 'category_0_1': [['โทรศัพท์มือถือและแท็บเล็ต'],
  ['smartphones'],
  ['โทรศัพท์มือถือ',
   'samsung',
   'galaxy',
   '50',
   's',
   'ram',

In [29]:
series = { key: pd.Series(val) for key, val in token.items() }

token2 = {}
for i,v in series.items():
    token2[i] = v.apply(pd.Series).stack()

In [30]:
df_processed = pd.DataFrame.from_dict(token2, orient='index').transpose().reset_index()

## Category 0

In [33]:
df_processed.filter(regex='category_0', axis=1)

,category_0_0,category_0_1,category_0_10,category_0_100,category_0_101,category_0_102,category_0_103,category_0_104,category_0_105,category_0_106,...,category_0_90,category_0_91,category_0_92,category_0_93,category_0_94,category_0_95,category_0_96,category_0_97,category_0_98,category_0_99
0,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,ผ่อน,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,...,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,apple,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,โทรศัพท์มือถือและแท็บเล็ต,ผ่อน,โทรศัพท์มือถือและแท็บเล็ต
1,smartphones,smartphones,smartphones,smartphones,smartphones,smartphones,smartphones,apple,smartphones,smartphones,...,smartphones,smartphones,ผ่อน,smartphones,smartphones,smartphones,smartphones,smartphones,โทรศัพท์มือถือ,smartphones
2,samsung,โทรศัพท์มือถือ,samsung,oppo,ผ่อน,ผ่อน,oppo,apple,oppo,โทรศัพท์มือถือ,...,ผ่อน,ผ่อน,apple,oppo,oppo,free,โทรศัพท์มือถือ,oppo,100,oppo
3,galaxy,samsung,galaxy,reno,0,0,9,iphone,f,oppo,...,0,0,iphone,3,3,gift,oppo,5,เปอร์เซนต์,5
4,m,galaxy,5,ram,เปอร์เซนต์,เปอร์เซนต์,2020,7,9,5,...,เปอร์เซนต์,เปอร์เซนต์,7,s,s,r,5,2020,สำหรับ,s
5,30,50,2017,6,ฟรีของแถม,มีของพร้อมส่ง,ram,32,NaN,2020,...,10,ฟรีของแถม,32,ใช้ได้ทุกเครือข่าย,2,15,2020,3,xiaomi,ram
6,s,s,NaN,256,oppo,oppo,8,gb,NaN,3,...,เดือน,oppo,gb,NaN,16,pro,ram,64,mi,3
7,4,ram,NaN,gb,5,reno,rom,โทรศัพท์,NaN,64,...,oppo,5,โทรศัพท์,NaN,gb,6,3,gb,9,32
8,64,4,NaN,snapdragon,s,ace,128,second,NaN,gb,...,9,2020,second,NaN,NaN,128,64,ประกันศูนย์,หน้าจอ,gb
9,gb,64,NaN,710,3,40,ประกันศูนย์,hand,NaN,ผ่อน,...,2020,3,hand,NaN,NaN,gb,gb,1,lcd,แถมหูฟัง


# Save data to csv file

In [39]:
# Transform dict to dataframe and save to csv file
pd.DataFrame.from_dict(data_clean, orient='index').transpose().reset_index().to_csv("./Data/{}_data-clean.csv".format(file_name))

In [140]:
pd.DataFrame.from_dict(dict_value, orient='index').transpose().to_csv("./Data/{}_value.csv".format(file_name))